# ATIVIDADE PRÁTICA - AULA 9 - REDES NEURAIS

# Comentários Gerais 
* Várias formas para separar B (Algodão, S/1) dos demais (N/0)
* 'Tunning':  
   - Grid: Valores ótimos não devem ser extremos de intervalo
   - Random: se range for grande n_iter não pode se pequeno
* Quando aparecer alguma mensagem relativa ao número máximo de iterações (max_iter) ter sido atingido e não ter ocorrido convergência, É **NECESSÁRIO AUMENTAR** *max_iter* e **RODAR NOVAMENTE**
* Em Redes Neurais:
   - 'max_iter' não é hiperparâmetro, mas condição de parada.   
      Na otimização de hiperparâmetros, esse não é um hiperparâmetro a ser otimizado.
   - Hiperparâmetros a serem priorizados para otimização:  
     - Número de camadas
     - Número de neurônios por camada
     - Taxa de aprendizagem (*'Learning Rate'*)
     - Função de ativação ('relu', 'adam')

* Em RandomForest, GBoost, XGBoost, etc.:  
   - Hiperparâmetros a serem priorizados para otimização:  
     - max_depth
     - n_estimators
     - max_fetaures

## Importar pacotes

In [4]:
# Bibliotecas Gerais
import pandas as pd
import numpy as np

# Bibliotecas para Preparação dos Dados
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

# Bibliotecas dos Modelos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

# Bibliotecas para Avaliação dos Modelos
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

# Imblearn (Imbalanced Learn)
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN

import tensorflow as tf

# Exercício 1 - MODIFICADO: Classificar E (Solo com Resíduos Vegetais)

*   Utilize o arquivo "landsat_full.csv"
*   Considere um problema para classificação de 'Solo Tipo III' (S) em contraposição a ‘Outros' (N)
*   Utilize as técnicas KNN, RandomForest e XGBoost

In [5]:
# Leitura do arquivo landsat_full.csv
df = pd.read_csv('landsat_full.csv')

In [6]:
# Verificar distribuição dos registros no atributo meta ID
df.ID.value_counts()

A    1533
F    1508
C    1358
E     707
B     703
D     626
Name: ID, dtype: int64

In [ ]:
# Várias formas de fazer a separação. Todas chegam a um resultado certo. Portanto, OK.
# Porém, algumas são mais ELEGANTES ('enxutas'). Vale a pena passar a usá-las.
# Vou mostrar algumas (não todas) que foram feitas nas Atividades Práticas.
# Em todas, vou converter para 1 (Sim) ou 0 (Não)

In [7]:
# 1)
# 'Solo Tipo III' é a classe 'D' no atributo meta ID. Com 626 registros (aprox. 9,7%)
# Converter todos registros em ID
df['ID1'] = df.ID.map({'A':0,
                       'B':0,
                       'C':0,
                       'D':1,
                       'E':0,
                       'F':0})

In [8]:
df.ID1.value_counts(0)

0    5809
1     626
Name: ID1, dtype: int64

In [9]:
# 2)
# 'Solo Tipo III' é a classe 'D' no atributo meta ID. Com 626 registros (aprox. 9,7%)
# Converter todos registros em ID
df['ID2']=df.ID
df['ID2'].where((df.ID2=='D'),0, inplace=True)
df['ID2'].where((df.ID2==0),1, inplace=True)

In [10]:
df.ID2.value_counts(0)

0    5809
1     626
Name: ID2, dtype: int64

In [11]:
# 3)
# 'Solo Tipo III' é a classe 'D' no atributo meta ID. Com 626 registros (aprox. 9,7%)
# Converter todos registros em ID
df3=df
df3=pd.get_dummies(df3)
df3=df3.drop(['ID_A','ID_B','ID_C','ID_E','ID_F'],axis=1)

In [12]:
df3.ID_D.value_counts()

0    5809
1     626
Name: ID_D, dtype: int64

In [13]:
# 4) 
# De forma mais compacta
df['ID4']=[1 if x == 'D' else 0 for x in df.ID]
#list comprehension

In [14]:
df.ID4.value_counts()

0    5809
1     626
Name: ID4, dtype: int64

In [15]:
# Vamos fazer novamente a leitura do arquivo landsat_full.csv
df = pd.read_csv('landsat_full.csv')
# E substituir ID == D por '1' e os demais, por '0'
df['ID']=[1 if x == 'D' else 0 for x in df.ID]

## Separação Atributos Preditores / Atributo Meta

In [16]:
# Atributo meta y
y = df.ID

# Atributos preditores X
X = df.drop(['ID'], axis=1)

## Treino/Teste com estratificação

In [17]:
# Treino (70%) / Teste (30%) estratificado pelo atributo y
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2020, test_size=0.3, stratify=y)

In [18]:
# Verificando dimensões dos conjuntos treino e teste
print('Dimensões dos Atributos Preditores X_train e X_test, e dos Atributos Meta y_train e y_test:')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Dimensões dos Atributos Preditores X_train e X_test, e dos Atributos Meta y_train e y_test:
(4504, 36) (1931, 36) (4504,) (1931,)


In [19]:
# Verificando proporções dos conjuntos treino e teste
print('Proporção dos valores do Atributo Meta: y_train')
print(y_train.value_counts(1))
print('Proporção dos valores do Atributo Meta: y_test')
print(y_test.value_counts(1))

Proporção dos valores do Atributo Meta: y_train
0    0.902753
1    0.097247
Name: ID, dtype: float64
Proporção dos valores do Atributo Meta: y_test
0    0.902641
1    0.097359
Name: ID, dtype: float64


## Normalização (Z-Score) dos dados

In [20]:
# Normalizar (Z-score) os dados

# Normalização do X_train
def_scaler = StandardScaler().fit(X_train)
X_train_norm = pd.DataFrame(pd.DataFrame(def_scaler.transform(X_train), columns=X_train.columns))

# Realizar mesma transformação em X_test
X_test_norm = pd.DataFrame(def_scaler.transform(X_test), columns=X_train.columns)

In [ ]:
# Verificação dos dados normalizados: X_train_norm
# De fato, a média é 0 e o desvio padrão 1?
X_train_norm.describe().transpose()

In [ ]:
# Verificação dos dados normalizados: X_test
# De fato, a média é 0 e o desvio padrão 1?
X_test_norm.describe().transpose()
#transformaçao feita usando conjunto de treino,
#media prox de 0, e prox de 1

## 1.A) Para cada um das técnicas e cada hiperparâmetro otimizado, apresente o intervalo (range) utilizado e o valor ótimo encontrado e utilizado para fazer a classificação. Construa uma tabela (DataFrame) semelhante ao quadro abaixo.
* OBS 1: Utilize RandomizedSearch do Scikit-Learn
* OBS 2: Para Neural Networks, considere todas camadas com o mesmo número de neurônios

### Otimização de Hiperparâmetros: RandomizedSerachCV

In [25]:
# Construção de uma função para 'hyperparameters tunning'
# REDUZ REPETIÇÃO DE DIGITAÇÃO!!!

def rnd_hyper_tunning(model, param, niter):
  rnd_search = RandomizedSearchCV(model, 
                                  param_distributions=param,
                                  n_iter=niter,
                                  cv=5,
                                  random_state=2020)
  rnd_search.fit(X_train_norm, y_train)
  return rnd_search.best_params_

## KNN

### SEM 'tunning' (KNN_1)

In [ ]:
# KNN sem 'tunning' (knn_1)
knn_1 = KNeighborsClassifier().fit(X_train_norm, y_train)
knn_1_pred = knn_1.predict(X_test_norm)
print(classification_report(y_test, knn_1_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97      1743
           1       0.68      0.69      0.68       188

    accuracy                           0.94      1931
   macro avg       0.82      0.83      0.82      1931
weighted avg       0.94      0.94      0.94      1931



### COM 'tunning'  (KNN_2)

In [ ]:
%%time
# KNN com 'tunning' (knn_2)
# Definir os parâmetros (dicionário) que se pretende otimizar
params = dict({'n_neighbors':np.arange(1,30)})
# Definir o número de vezes que vai rodar combinando os hiperparâmetros conforme definido acima
niter = 15

# Chamar a função rnd_hyper_tunning
knn_2_params = rnd_hyper_tunning(KNeighborsClassifier(), params,niter)
# Mostrar os melhores hiperparâmetros
print("Melhor K", knn_2_params)

Melhor K {'n_neighbors': 2}
CPU times: user 11.1 s, sys: 1.26 ms, total: 11.1 s
Wall time: 11.1 s


In [ ]:
knn_2 = KNeighborsClassifier(n_neighbors=knn_2_params['n_neighbors']).fit(X_train_norm, y_train)
knn_2_pred = knn_2.predict(X_test_norm)
print(classification_report(y_test, knn_2_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      1743
           1       0.78      0.57      0.66       188

    accuracy                           0.94      1931
   macro avg       0.87      0.78      0.81      1931
weighted avg       0.94      0.94      0.94      1931



## Random Forest

### SEM 'tunning' (rf_1)

In [ ]:
%%time
# RandomForest sem 'tunning' (rf_1)
with tf.device('/device:GPU:0'):
  rf_1 = RandomForestClassifier(random_state=2020).fit(X_train_norm, y_train)
  rf_1_pred = rf_1.predict(X_test_norm)

print(classification_report(y_test, rf_1_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1743
           1       0.86      0.55      0.67       188

    accuracy                           0.95      1931
   macro avg       0.91      0.77      0.82      1931
weighted avg       0.94      0.95      0.94      1931

CPU times: user 740 ms, sys: 0 ns, total: 740 ms
Wall time: 741 ms


### COM 'tunning' (rf_2)

In [ ]:
%%time
with tf.device('/device:GPU:0'):
# RandomForest com 'tunning' (rf_2)
# Definir os parâmetros (dicionário) que se pretende otimizar
  params = dict({'max_features':np.linspace(0.1, 0.8, 8),
               'n_estimators':np.arange(500,1501,100)})
# Definir o número de vezes que vai rodar combinando os hiperparâmetros conforme definido acima
  niter = 30
# Chamar a função rnd_hyper_tunning
  rf_2_params = rnd_hyper_tunning(RandomForestClassifier(random_state=2020), params,niter)

# Mostrar os melhores hiperparâmetros
print("Melhores parametros para RandomForest", rf_2_params)

Melhores parametros para RandomForest {'n_estimators': 1500, 'max_features': 0.4}
CPU times: user 30min 6s, sys: 1.61 s, total: 30min 8s
Wall time: 30min 8s


In [ ]:
%%time
with tf.device('/device:GPU:0'):
  rf_2 = RandomForestClassifier(random_state=2020, 
                                max_features = rf_2_params['max_features'],
                                n_estimators = rf_2_params['n_estimators']).fit(X_train_norm, y_train)
  rf_2_pred = rf_2.predict(X_test_norm)


print(classification_report(y_test, rf_2_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1743
           1       0.86      0.56      0.68       188

    accuracy                           0.95      1931
   macro avg       0.91      0.77      0.82      1931
weighted avg       0.95      0.95      0.94      1931

CPU times: user 20.6 s, sys: 17 ms, total: 20.7 s
Wall time: 20.7 s


## XGBoost

### SEM 'tunning' (xgb_1)


In [ ]:
%%time
# XGBoost sem 'tunning' (xgb_1)
with tf.device('/device:GPU:0'):
  xgb_1 = XGBClassifier(random_state=2020).fit(X_train_norm, y_train)
  xgb_1_pred = xgb_1.predict(X_test_norm)

print(classification_report(y_test, xgb_1_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      1743
           1       0.78      0.53      0.63       188

    accuracy                           0.94      1931
   macro avg       0.86      0.76      0.80      1931
weighted avg       0.93      0.94      0.93      1931

CPU times: user 459 ms, sys: 3.88 ms, total: 462 ms
Wall time: 462 ms


### COM 'tunning' (xgb_2)

In [ ]:
%%time
with tf.device('/device:GPU:0'):
# RandomForest com 'tunning' (rf_2)
# Definir os parâmetros (dicionário) que se pretende otimizar
  params = dict({'max_depth':np.arange(3,21,1),
                 'n_estimators':np.arange(500,1501,100)})
# Definir o número de vezes que vai rodar combinando os hiperparâmetros conforme definido acima
  niter = 30
# Chamar a função rnd_hyper_tunning
  xgb_2_params = rnd_hyper_tunning(RandomForestClassifier(random_state=2020), params,niter)

# Mostrar os melhores hiperparâmetros
print("Melhores parametros para RandomForest", xgb_2_params)

Melhores parametros para RandomForest {'n_estimators': 600, 'max_depth': 16}
CPU times: user 13min 30s, sys: 1.28 s, total: 13min 32s
Wall time: 13min 31s


In [ ]:
%%time
with tf.device('/device:GPU:0'):
  xgb_2 = XGBClassifier(random_state=2020,
                        max_depth = xgb_2_params['max_depth'],
                        n_estimators = xgb_2_params['n_estimators']).fit(X_train_norm, y_train)
  xgb_2_pred = xgb_2.predict(X_test_norm)

print(classification_report(y_test, xgb_2_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1743
           1       0.84      0.65      0.73       188

    accuracy                           0.95      1931
   macro avg       0.90      0.82      0.85      1931
weighted avg       0.95      0.95      0.95      1931

CPU times: user 6.61 s, sys: 24 ms, total: 6.63 s
Wall time: 6.63 s


## DataFrame com os Hiperparâmetros

In [ ]:
df_HiperP_Otim = pd.DataFrame([['KNN', 'n_neighbors',1,30,2],
                               ['Random Forest','max_features', 0.1,0.8,0.4],
                               ['Random Forest','n_estimators', 500,1500,1500],
                               ['XGBoost','max_depth', 3,21,16],
                               ['XGBoost','n_estimators', 500,1500,600]],
                              columns = ['Técnica', 'Hiperparâmetro', 'Min', 'Max', 'Ótimo'])
df_HiperP_Otim

,Técnica,Hiperparâmetro,Min,Max,Ótimo
0,KNN,n_neighbors,1.0,30.0,2.0
1,Random Forest,max_features,0.1,0.8,0.4
2,Random Forest,n_estimators,500.0,1500.0,1500.0
3,XGBoost,max_depth,3.0,21.0,16.0
4,XGBoost,n_estimators,500.0,1500.0,600.0


### DISCUTIR CONCEITOS
* Resultados significativamente diferentes obtidos com **GRIDSearch** e **RANDOMSearch**
* Ótimo no extremo do intervalo

## DataFrame com a compilação dos resultados: KNN, RandomForest e XGBoost

In [ ]:
# Acurácia (acc)
knn_1_acc = accuracy_score(y_test, knn_1_pred)
knn_2_acc = accuracy_score(y_test, knn_2_pred)

rf_1_acc = accuracy_score(y_test, rf_1_pred)
rf_2_acc = accuracy_score(y_test, rf_2_pred)

xgb_1_acc = accuracy_score(y_test, xgb_1_pred)
xgb_2_acc = accuracy_score(y_test, xgb_2_pred)

In [ ]:
# Precisão (prec)
knn_1_prec_N, knn_1_prec_S = precision_score(y_test, knn_1_pred, average=None)
knn_2_prec_N, knn_2_prec_S = precision_score(y_test, knn_2_pred, average=None)

rf_1_prec_N,rf_1_prec_S = precision_score(y_test, rf_1_pred, average=None)
rf_2_prec_N,rf_2_prec_S = precision_score(y_test, rf_2_pred, average=None)

xgb_1_prec_N, xgb_1_prec_S = precision_score(y_test, xgb_1_pred, average=None)
xgb_2_prec_N, xgb_2_prec_S = precision_score(y_test, xgb_2_pred, average=None)

In [ ]:
# Recall (recall)
knn_1_recall_N, knn_1_recall_S = recall_score(y_test, knn_1_pred, average=None)
knn_2_recall_N, knn_2_recall_S = recall_score(y_test, knn_2_pred, average=None)

rf_1_recall_N,rf_1_recall_S = recall_score(y_test, rf_1_pred, average=None)
rf_2_recall_N,rf_2_recall_S = recall_score(y_test, rf_2_pred, average=None)

xgb_1_recall_N, xgb_1_recall_S = recall_score(y_test, xgb_1_pred, average=None)
xgb_2_recall_N, xgb_2_recall_S = recall_score(y_test, xgb_2_pred, average=None)

In [ ]:
# f-1 Score (f1)
knn_1_f1_N, knn_1_f1_S = f1_score(y_test, knn_1_pred, average=None)
knn_2_f1_N, knn_2_f1_S = f1_score(y_test, knn_2_pred, average=None)

rf_1_f1_N,rf_1_f1_S = f1_score(y_test, rf_1_pred, average=None)
rf_2_f1_N,rf_2_f1_S = f1_score(y_test, rf_2_pred, average=None)

xgb_1_f1_N, xgb_1_f1_S = f1_score(y_test, xgb_1_pred, average=None)
xgb_2_f1_N, xgb_2_f1_S = f1_score(y_test, xgb_2_pred, average=None)

In [ ]:
# Criação do DataFrame 'result_geral'
cols = ['Metrica','KNN_1','KNN_2','RandomForest_1','RandomForest_2','XGBoost_1','XGBoost_2']
result_geral = pd.DataFrame(columns=cols)
result_geral.Metrica = ['Acuracia', 'Precisao (S)', 'Recall (S)', 'f-1 Score (S)']
result_geral.KNN_1 = [knn_1_acc,knn_1_prec_S, knn_1_recall_S,knn_1_f1_S]
result_geral.KNN_2 = [knn_2_acc,knn_2_prec_S, knn_2_recall_S,knn_2_f1_S]
result_geral.RandomForest_1 = [rf_1_acc,rf_1_prec_S, rf_1_recall_S,rf_1_f1_S]
result_geral.RandomForest_2 = [rf_2_acc,rf_2_prec_S, rf_2_recall_S,rf_2_f1_S]
result_geral.XGBoost_1 = [xgb_1_acc,xgb_1_prec_S, xgb_1_recall_S,xgb_1_f1_S]
result_geral.XGBoost_2 = [xgb_2_acc,xgb_2_prec_S, xgb_2_recall_S,xgb_2_f1_S]

In [ ]:
result_geral

,Metrica,KNN_1,KNN_2,RandomForest_1,RandomForest_2,XGBoost_1,XGBoost_2
0,Acuracia,0.937856,0.941999,0.947695,0.948213,0.939410,0.953910
1,Precisao (S),0.677083,0.775362,0.859504,0.860656,0.775194,0.841379
2,Recall (S),0.691489,0.569149,0.553191,0.558511,0.531915,0.648936
3,f-1 Score (S),0.684211,0.656442,0.673139,0.677419,0.630915,0.732733


# Exercício 2 - MODIFICADO: Classificar E (Solo com Resíduos Vegetais) - Balanceando o conjunto de dados

*   Utilize o arquivo "landsat_full.csv"
*   Considere um problema para classificação de 'Solo Tipo III' (S) em contraposição a ‘Outros' (N)
*   Utilize as técnicas KNN, RandomForest e XGBoost

## Leitura e preprocessamento dos dados
* Atributos Preditores / Atributo Meta
* Conjuntos Treino / Teste
* Normalização (Z-Score)

In [ ]:
# Leitura do arquivo landsat_full.csv
df = pd.read_csv('landsat_full.csv')

# Conversão no atributo meta ID: 'Solo Tipo III' (classe 'D') para '1' e demais registros para '0'
df['ID']=[1 if x=='D' else 0 for x in df.ID]

# Separação Atributos Preditores / Atributo Meta

# Atributo meta y
y = df.ID

# Atributos preditores X
X = df.drop(['ID'], axis=1)

# Construção de Treino/Teste com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2020, test_size=0.3, stratify=y)

# Normalização (Z-score) dos dados

# Normalização do X_train
def_scaler = StandardScaler().fit(X_train)
X_train_norm = pd.DataFrame(pd.DataFrame(def_scaler.transform(X_train), columns=X_train.columns))

# Realizar mesma transformação em X_test
X_test_norm = pd.DataFrame(def_scaler.transform(X_test), columns=X_train.columns)

## Balanceamento dos Dados
* SMOTE
* Tomek Links
* SMOTETomek
* ENN
* SMOTEEnn

### SMOTE

In [ ]:
%%time
# Definir o modelo (smt = SMOTE)
smt = SMOTE(random_state=2020)

# Construir X_train_smt e y_train_smt
X_train_smt, y_train_smt = smt.fit_resample(X_train_norm,y_train)

# Transformar X_train_smt em DataFrame com os nomes originais
X_train_smt = pd.DataFrame(X_train_smt, columns=X.columns)

# Transformar y_train_smt em Series
y_train_smt = pd.Series(y_train_smt)

CPU times: user 37.1 ms, sys: 3.01 ms, total: 40.1 ms
Wall time: 40.8 ms


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Verificar distibuição de y (original) e y_train_smt
print('Distribuição y_train(original)')
print(y_train.value_counts())

print('Distribuição y_train_smt')
print(y_train_smt.value_counts())

Distribuição y_train(original)
0    4066
1     438
Name: ID, dtype: int64
Distribuição y_train_smt
1    4066
0    4066
dtype: int64


### Tomek Links

In [ ]:
%%time
# Definir o modelo (smt = SMOTE)
tmk_link = TomekLinks(random_state=2020)

# Construir X_train_smt e y_train_smt
X_train_tmk, y_train_tmk = tmk_link.fit_resample(X_train_norm,y_train)

# Transformar X_train_smt em DataFrame com os nomes originais
X_train_tmk = pd.DataFrame(X_train_tmk, columns=X.columns)

# Transformar y_train_smt em Series
y_train_tmk = pd.Series(y_train_tmk)

CPU times: user 500 ms, sys: 742 µs, total: 501 ms
Wall time: 503 ms


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Verificar distibuição de y (original) e y_train_tmk
print('Distribuição y_train(original)')
print(y_train.value_counts())

print('Distribuição y_train_tmk')
print(y_train_tmk.value_counts())

Distribuição y_train(original)
0    4066
1     438
Name: ID, dtype: int64
Distribuição y_train_tmk
0    4036
1     438
dtype: int64


### SMOTETomek

In [ ]:
%%time
# Definir o modelo (smt_tmk = SMOTETomek)
smt_tmk = SMOTETomek(random_state=2020)

# Construir X_train_smt e y_train_smt
X_train_smt_tmk, y_train_smt_tmk = smt_tmk.fit_resample(X_train_norm,y_train)

# Transformar X_train_smt em DataFrame com os nomes originais
X_train_smt_tmk = pd.DataFrame(X_train_smt_tmk, columns=X.columns)

# Transformar y_train_smt em Series
y_train_smt_tmk = pd.Series(y_train_smt_tmk)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


CPU times: user 807 ms, sys: 1.8 ms, total: 809 ms
Wall time: 811 ms


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Verificar distibuição de y (original) e y_train_smt_tmk
print('Distribuição y_train(original)')
print(y_train.value_counts())

print('Distribuição y_train_smt_tmk')
print(y_train_smt_tmk.value_counts())

Distribuição y_train(original)
0    4066
1     438
Name: ID, dtype: int64
Distribuição y_train_smt_tmk
1    4066
0    4066
dtype: int64


### ENN

In [ ]:
%%time
# Definir o modelo (enn = EditedNearestNeighbours)
enn = EditedNearestNeighbours(random_state=2020)

# Construir X_train_smt e y_train_smt
X_train_enn, y_train_enn = enn.fit_resample(X_train_norm,y_train)

# Transformar X_train_smt em DataFrame com os nomes originais
X_train_enn = pd.DataFrame(X_train_enn, columns=X.columns)

# Transformar y_train_smt em Series
y_train_enn = pd.Series(y_train_enn)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


CPU times: user 489 ms, sys: 1.01 ms, total: 490 ms
Wall time: 496 ms


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Verificar distibuição de y (original) e y_train_enn
print('Distribuição y_train(original)')
print(y_train.value_counts())

print('Distribuição y_train_enn')
print(y_train_enn.value_counts())

Distribuição y_train(original)
0    4066
1     438
Name: ID, dtype: int64
Distribuição y_train_enn
0    3693
1     438
dtype: int64


### SMOTEENN

In [ ]:
%%time
# Definir o modelo (smt_enn = SMOTEENN)
smt_enn = SMOTEENN(random_state=2020)

# Construir X_train_smt e y_train_smt
X_train_smt_enn, y_train_smt_enn = smt_enn.fit_resample(X_train_norm,y_train)

# Transformar X_train_smt em DataFrame com os nomes originais
X_train_smt_enn = pd.DataFrame(X_train_smt_enn, columns=X.columns)

# Transformar y_train_smt em Series
y_train_smt_enn = pd.Series(y_train_smt_enn)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


CPU times: user 1.06 s, sys: 5.83 ms, total: 1.07 s
Wall time: 1.07 s


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Verificar distibuição de y (original) e y_train_smt_enn
print('Distribuição y_train(original)')
print(y_train.value_counts())

print('Distribuição y_train_smt_enn')
print(y_train_smt_enn.value_counts())

Distribuição y_train(original)
0    4066
1     438
Name: ID, dtype: int64
Distribuição y_train_smt_enn
1    4065
0    3424
dtype: int64


## KNN - Com dados normalizados (Z-Score), balanceados e 'tunning' do hiperparâmetros

### KNN SMOTE

In [ ]:
%%time
# KNN com 'tunning' e dados balanceados (knn_bal)
# Definir os parâmetros (dicionário) que se pretende otimizar
params = dict({'n_neighbors':np.arange(1,30)})
# Definir o número de vezes que vai rodar combinando os hiperparâmetros conforme definido acima
niter = 15

rnd_search = RandomizedSearchCV(KNeighborsClassifier(),
                                param_distributions=params,
                                n_iter=niter,
                                cv=5,
                                random_state=2020)

rnd_search.fit(X_train_smt, y_train_smt)

knn_smt_opt = rnd_search.best_params_
print('Melhor K(SMOTE): ', knn_smt_opt)

Melhor K(SMOTE):  {'n_neighbors': 2}
CPU times: user 27.2 s, sys: 15.3 ms, total: 27.2 s
Wall time: 27.2 s


In [ ]:
knn_smt = KNeighborsClassifier(n_neighbors=knn_smt_opt['n_neighbors']).fit(X_train_smt, y_train_smt)
knn_smt_pred = knn_smt.predict(X_test_norm)
print(classification_report(y_test, knn_smt_pred))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      1743
           1       0.58      0.82      0.68       188

    accuracy                           0.93      1931
   macro avg       0.78      0.88      0.82      1931
weighted avg       0.94      0.93      0.93      1931



### KNN Tomek Link

In [ ]:
%%time
# KNN com 'tunning' e dados balanceados (knn_bal)
# Definir os parâmetros (dicionário) que se pretende otimizar
params = dict({'n_neighbors':np.arange(1,30)})
# Definir o número de vezes que vai rodar combinando os hiperparâmetros conforme definido acima
niter = 15

rnd_search = RandomizedSearchCV(KNeighborsClassifier(),
                                param_distributions=params,
                                n_iter=niter,
                                cv=5,
                                random_state=2020)

rnd_search.fit(X_train_tmk, y_train_tmk)

knn_tmk_opt = rnd_search.best_params_
print('Melhor K(Tomek Link): ', knn_tmk_opt)

Melhor K(Tomek Link):  {'n_neighbors': 2}
CPU times: user 11.1 s, sys: 5.93 ms, total: 11.1 s
Wall time: 11.1 s


In [ ]:
knn_tmk = KNeighborsClassifier(n_neighbors=knn_tmk_opt['n_neighbors']).fit(X_train_tmk, y_train_tmk)
knn_tmk_pred = knn_tmk.predict(X_test_norm)
print(classification_report(y_test, knn_tmk_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1743
           1       0.76      0.59      0.66       188

    accuracy                           0.94      1931
   macro avg       0.86      0.78      0.81      1931
weighted avg       0.94      0.94      0.94      1931



### KNN SMOTETomek

In [ ]:
%%time
# KNN com 'tunning' e dados balanceados (knn_bal)
# Definir os parâmetros (dicionário) que se pretende otimizar
params = dict({'n_neighbors':np.arange(1,30)})
# Definir o número de vezes que vai rodar combinando os hiperparâmetros conforme definido acima
niter = 15

rnd_search = RandomizedSearchCV(KNeighborsClassifier(),
                                param_distributions=params,
                                n_iter=niter,
                                cv=5,
                                random_state=2020)

rnd_search.fit(X_train_smt_tmk, y_train_smt_tmk)

knn_smt_tmk_opt = rnd_search.best_params_
print('Melhor K(SMOTETomek): ', knn_smt_tmk_opt)

Melhor K(SMOTETomek):  {'n_neighbors': 2}
CPU times: user 27 s, sys: 19.9 ms, total: 27.1 s
Wall time: 27.1 s


In [ ]:
knn_smt_tmk = KNeighborsClassifier(n_neighbors=knn_smt_tmk_opt['n_neighbors']).fit(X_train_smt_tmk, y_train_smt_tmk)
knn_smt_tmk_pred = knn_smt_tmk.predict(X_test_norm)
print(classification_report(y_test, knn_smt_tmk_pred))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      1743
           1       0.58      0.82      0.68       188

    accuracy                           0.93      1931
   macro avg       0.78      0.88      0.82      1931
weighted avg       0.94      0.93      0.93      1931



### KNN ENN

In [ ]:
%%time
# KNN com 'tunning' e dados balanceados (knn_bal)
# Definir os parâmetros (dicionário) que se pretende otimizar
params = dict({'n_neighbors':np.arange(1,30)})
# Definir o número de vezes que vai rodar combinando os hiperparâmetros conforme definido acima
niter = 15

rnd_search = RandomizedSearchCV(KNeighborsClassifier(),
                                param_distributions=params,
                                n_iter=niter,
                                cv=5,
                                random_state=2020)

rnd_search.fit(X_train_enn, y_train_enn)

knn_enn_opt = rnd_search.best_params_
print('Melhor K(ENN): ', knn_enn_opt)

Melhor K(ENN):  {'n_neighbors': 3}
CPU times: user 9.62 s, sys: 10.9 ms, total: 9.63 s
Wall time: 9.65 s


In [ ]:
knn_enn = KNeighborsClassifier(n_neighbors=knn_enn_opt['n_neighbors']).fit(X_train_enn, y_train_enn)
knn_enn_pred = knn_tmk.predict(X_test_norm)
print(classification_report(y_test, knn_enn_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1743
           1       0.76      0.59      0.66       188

    accuracy                           0.94      1931
   macro avg       0.86      0.78      0.81      1931
weighted avg       0.94      0.94      0.94      1931



### KNN SMOTEENN



In [ ]:
%%time
# KNN com 'tunning' e dados balanceados (knn_bal)
# Definir os parâmetros (dicionário) que se pretende otimizar
params = dict({'n_neighbors':np.arange(1,30)})
# Definir o número de vezes que vai rodar combinando os hiperparâmetros conforme definido acima
niter = 15

rnd_search = RandomizedSearchCV(KNeighborsClassifier(),
                                param_distributions=params,
                                n_iter=niter,
                                cv=5,
                                random_state=2020)

rnd_search.fit(X_train_smt_enn, y_train_smt_enn)

knn_smt_enn_opt = rnd_search.best_params_
print('Melhor K(SMOTETomek): ', knn_smt_enn_opt)

Melhor K(SMOTETomek):  {'n_neighbors': 2}
CPU times: user 22.8 s, sys: 13.9 ms, total: 22.8 s
Wall time: 22.8 s


In [ ]:
knn_enn_tmk = KNeighborsClassifier(n_neighbors=knn_smt_enn_opt['n_neighbors']).fit(X_train_smt_enn, y_train_smt_enn)
knn_enn_tmk_pred = knn_enn_tmk.predict(X_test_norm)
print(classification_report(y_test, knn_enn_tmk_pred))

              precision    recall  f1-score   support

           0       0.99      0.89      0.94      1743
           1       0.48      0.93      0.63       188

    accuracy                           0.89      1931
   macro avg       0.73      0.91      0.79      1931
weighted avg       0.94      0.89      0.91      1931

